In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.causal_model import *
from src.lime_explainer import * 
from src.calime_explainer import *
from src.experiments.plausibility import *

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


import warnings
warnings.simplefilter(action='ignore')

In [ ]:
dataset_path = './data/banknote.csv'

In [ ]:
df = pd.read_csv(dataset_path, index_col=False)

In [ ]:
# Shuffle the data to avoid that last samples have always the same class
df = df.sample(len(df))

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

features_names = list(X.columns)
class_values = list(y.unique())

In [ ]:
num_samples = 100

In [ ]:
# train and test set
train, test = X.head(len(X) - num_samples), X.tail(num_samples)
# save test 
#test.to_csv(path + '/test.csv', index_label='index')
labels_train, labels_test = y.head(len(y) - num_samples), y.tail(num_samples)

train_bb, train_gen, y_bb, y_gen = train_test_split(train, labels_train, test_size=0.3, random_state=0)

In [ ]:
path = '/Users/martina/Desktop/'

In [ ]:
generative_model, graph = get_causal_model(train_gen, features_names, path)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
estimator = RandomForestClassifier()
hyper_param = dict(n_estimators=[100, 300, 500, 1000],
                        max_depth=[5, 8, 15],
                        min_samples_split=[2, 5, 10, 15, 100],
                        min_samples_leaf=[1, 2, 5, 10]
                       )

In [ ]:
search = RandomizedSearchCV(estimator, hyper_param)
search.fit(train_bb, y_bb)
bb = search.best_estimator_

In [ ]:
y_pred = bb.predict(test)
#filename = 'bb_model.sav'
#pickle.dump(bb, open(path '/' + filename, 'wb'))

In [ ]:
print(classification_report(labels_test, y_pred, labels=class_values))

In [ ]:
test = test.reset_index()

In [ ]:
row = test.sample(n=1)
data_row = row.iloc[:, 1:].values.flatten()
    
print('index:', test.iloc[row.index[0], 0])
print('data_row: ', data_row)

## LIME

In [ ]:
lime_explainer = LimeTabularExplainer(train_bb.values, feature_names=features_names,
                                      class_names=class_values, discretize_continuous=False)

lime_exp, lime_data, lime_neighbor_gen_time = lime_explainer.explain_instance(data_row, bb.predict_proba, 
                                                                              num_samples=1000)

## CALIME

In [ ]:
calime_explainer = CALimeTabularExplainer(train_bb.values, feature_names=features_names,
                                          class_names=class_values, discretize_continuous=False)

calime_exp, calime_data, calime_neighbor_gen_time = calime_explainer.explain_instance(data_row,
                                                                                     bb.predict_proba,
                                                                                     graph, generative_model)

In [ ]:
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime.lime_tabular
from __future__ import print_function
np.random.seed(1)

In [ ]:
iris = sklearn.datasets.load_iris()

In [ ]:
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(iris.data, iris.target, train_size=0.80)

In [ ]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train)

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(train_bb.values, feature_names=features_names, class_names=class_values
                                                   , discretize_continuous=True)

In [ ]:
i = np.random.randint(0, test.shape[0])
exp = explainer.explain_instance(test[i], rf.predict_proba, num_features=2, top_labels=1)

In [ ]:
test[i].shape

In [ ]:
train_bb.values.shape

In [7]:
from math import dist

In [11]:
def AMD (og, new): #
    
    og = og.values.tolist()
    new = new.values.tolist()
    
    min_distances = []
    
    for record in new:
        distances = []
        for og_record in og:
            d = dist (record, og_record)
            distances.append(d)

        min_distances.append(min(distances))

    return statistics.mean(min_distances)

In [12]:
df_genetic = pd.read_json("/Users/martina/Downloads/df_genetic.json")
df_fair = pd.read_json("/Users/martina/Downloads/df_fair.json")
df_adult = pd.read_json("/Users/martina/Downloads/df_adult.json")

In [13]:
AMD(df_adult, df_genetic)

NameError: name 'statistics' is not defined